In [15]:
from urllib.request import urlopen, Request
from os import mkdir
from os.path import exists
from bs4 import BeautifulSoup as bs
from base64 import b64encode
import re

def open_url(url, headers={}, auth=None, encoding='utf-8', retry=3):
    if auth != None and isinstance(auth, str): headers['Authorization'] = auth
    headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
    while retry > 0 or retry == -1:
        try: return bs(urlopen(Request(url, headers=headers)).read().decode(encoding), 'html.parser')
        except Exception as e: print('Retrying "{}" in cause of URLError: {}'.format(url, e.reason))
        if retry == -1: continue
        retry -= 1
        
def basic_auth(username, password):
    return 'Basic {}'.format(str(b64encode('{}:{}'.format(username, password).encode('utf-8')))[2:-1])

def text_matcher(regex):
    return lambda text: re.match(regex, text)

def has_key(key):
    def inner(obj):
        try: return True if obj[key] else False
        except KeyError: return False
    return inner

def get_key(key):
    return lambda obj: obj[key] if has_key(key)(obj) else None

def get_article(url, corpus, filename, title):
    file = 'articles/{}'.format(corpus)
    if not exists(file): mkdir(file)
    with open('{}/{}'.format(file, filename), 'w', encoding='utf-8') as file:
        article = ' '.join(filter(lambda x: len(x) > 30, map(lambda x: x.text, open_url(url, retry=-1, auth=basic_auth('nstp', 'iamnstp')).find_all('p')[:-1])))
        file.write(article)
        print('Got article: {}'.format(title))
        return url, title, article

def get_article_and_save(query, page):
    url = 'https://www.bharian.com.my/search?s={}{}'.format(query, '' if page == 0 else '&page={}'.format(page))
    regex = r'^/(.+?/.+?)/(\d{4}/\d{2})/(\d+?)/(.+)$'
    for url in set(filter(text_matcher(regex), map(get_key('href'), filter(has_key('href'), open_url(url, retry=-1).find_all('a'))))):
        corpus, date, id, title = text_matcher(regex)(url).groups()
        if len(corpus.split('/')) != 2: continue
        corpus = corpus.replace('/', '-')
        title = re.sub(r'%\d{2}', '', title)
        url = 'https://staging.bharian.com.my{}'.format(url)
        filename = '{}_{}_{}.txt'.format(date.replace('/', '-'), id, title)
        if exists('articles/{}/{}'.format(corpus, filename)): 
            print('Found existing article: {}'.format(title))
            continue
        yield get_article(url, corpus, filename, title)

def crawl(query, page=0):
    while page < 21: # 21 pages each query (including page 0)
        yield from get_article_and_save(query=query, page=page)
        page += 1

def crawler():
    letters = 'abcdefghijklmnopqrstuvwxyz'
    for i in letters:
        for j in letters:
            for k in letters:
                yield from crawl(query=''.join([i, j, k]))

In [ ]:
if not exists('articles'): mkdir('articles')
for url, title, article in crawler():
    pass

Found existing article: jangan-sampai-saya-ambil-tindakan-undang-undang
Found existing article: elakkan-aktiviti-riadah-di-tempat-berisiko-pada-musim-hujan
Found existing article: motif-serangan-pos-kawalan-disiasat
Found existing article: patah-kaki-tersepit-eskalator
Found existing article: nadal-rampas-ranking-nombor-satu-dunia
Found existing article: filem-upin-ipin-bawa-cabaran-negara-di-aaa-2019
Found existing article: dr-mahathir-optimis-pemulihan-negara
Found existing article: pochettino-mahu%C2%A0alli-kembali-nakal
Found existing article: strategi-lampard-di-chelsea-buah-hasil
Found existing article: larian-amal-bantu-golongan-memerlukan
Found existing article: real-lepas-peluang-pintas-barca
Found existing article: dbp-kaji-semula-pemilihan-sasterawan-negara
Found existing article: maut-rempuh-belakang-lori-rosak
Found existing article: mekanik-ditahan-rogol-kanak-kanak-hingga-hamil
Found existing article: agong-lawat-karnival-usahawan-koperasi
Found existing article: ramasam

Found existing article: ketum-lebih-200kg-dirampas
Found existing article: gelombang-haba-eropah-dijangka-semakin-teruk
Found existing article: pendaftaran-pemilih-dengan-berswafoto-bersama-kad-pengenalan
Found existing article: 19-maut-45-cedera-serangan-udara-russia-di-syria
Found existing article: piiacuf-berupaya-jadi-landasan-penyatuan-ummah
Found existing article: tender-terbuka-kompetitif-belanjawan-asas-sifar-beri-kesan-positif
Found existing article: hanya-gambar-kumpulan-tiada-rombakan-kabinet-pm
Found existing article: hafizh-tak-hilang-fokus
Found existing article: red-giants-menang-tipis-atasi-pkns-fc
Found existing article: ctos-beli-16-peratus-kepentingan-ramci
Found existing article: jurukamera-sambilan-rtm-menang-saman-terima-rm114470
Found existing article: malaysia-gerbang-ekonomi-halal
Found existing article: lisa-surihani-perjuang-hak-anak-tak-sah-taraf
Found existing article: kerajaan-perlu-tegas-tangani-arak-didakwa-tanpa-alkohol-mufti
Found existing article: atl

Got article: singkir-tun-m-sebagai-pm-satu-penghinaan-rina-harun
Got article: musim-akhir-game-throne-ditayang-mulai-14-april
Got article: polis-serbu-pusat-karaoke-tahan-22-gro-warga-asing
Got article: sarawak-buat-tfc-ii-kempunan
Got article: pemilik-tangkap-sendiri-pencuri-motosikal
Got article: lapan-pendayung-sertai-ekspedisi-kayak-3-hari
Got article: dewan-rakyat-lulus-noraini-pengerusi-pac-baharu
Got article: penjualan-tanah-ltat-tak-patuhi-piawaian
Got article: etnik-rohingya-cari-peluang-kawasan-tak-berpagar-aksem-kedah
Got article: arsenal-atasi-napoli
Got article: bursa-malaysia-dibuka-tinggi
Got article: jawatankuasa-masjid-melaka-pertama-laksana-ibr
Got article: giliran-korea-selatan-pula-jinak-skuad-negara
Got article: dr-streram-akan-berusaha-tambah-baik-klinik-kesihatan-rantau
Got article: kajian-menyeluruh-isu-gaji-siswazah-tpm
Got article: nyamuk-disuntik-wolbachia-atasi-wabak-denggi
Got article: keputusan-3-aksi-kelayakan-b-23-jadi-persoalan
Got article: tabung-legen

Got article: tpuns-bantu-1000-usahawan-kecil
Got article: jkkp-terengganu-keluar-53-notis-kesalahan-di-tapak-binaan
Got article: pindaan-mtn-mudahkan-urusan-bahagi-pusaka
Got article: polis-tahan-11-individu-terbabit-daesh-di-terengganu-sejak-2013
Got article: harga-minyak-kembali-stabil
Got article: pdrm-interpol-polis-india-bekerjasama-mencari-habibuneesa
Got article: pbb-gesa-sekat-perniagaan-tentera-myanmar-firma-asing-bersubahat
Got article: lembu-untuk-korban-mencukupi
Got article: man-city-ikat-cancelo-danilo-sertai-juve
Got article: china-beri-amaran-india-jangan-sekat-huawei
Got article: mpm-tegas-siasat-dakwaan-sindiket-jual-laporan-perubatan
Got article: katy-perry-syarikat-rakaman-diarah-bayar-rm112-juta
Got article: dulu-asnaf-kini-jutawan-kosmetik
Got article: jualan-sektor-pembuatan-naik-53-peratus
Got article: kedudukan-mat-zan-jadi-tanda-tanya
Got article: maric-rancang-kerjasama-dengan-mrl-cccc
Got article: analisis-kilang-berisiko-pasir-gudang-siap-bulan-depan
Got ar

Got article: pakai-apron-cerah
Got article: permintaan-untuk-aksesori-mutiara-meningkat
Got article: itu-gambar-lama-kami-kenapa-baru-nak-viral
Got article: alat-khas-percepat-proses-pembuatan-lemang
Got article: rhb-turunkan-br-blr
Got article: tiga-pemancing-tempuh-detik-cemas-bot-terbalik
Got article: gempar-ditimpa-hujan-air-batu
Got article: tiada-gelombang-haba-di-sabah
Got article: pertandingan-5-penjuru-pada-prk-parlimen-sandakan
Got article: mungkin-ke-pentas-teater-10-juta-tonton-nur-2
Got article: polis-buru-yamcik-disyaki-jual-telefon-bimbit-curi
Got article: 55-pelajar-uitm-santuni-gelandangan-jalan-tar
Got article: lori-terbabas-masuk-landasan-kereta-api
Got article: disingkir-psg-idola-spurs
Got article: dron-iran-pantau-pergerakan-kapal-di-teluk
Got article: kpkt-bincang-status-flet-keramat-permai
Got article: uchee-berlakon-filem-komedi
Got article: pelajar-ump-terima-wang-saku-sumbangan-agrobank
Got article: sri-lanka-larang-penduduk-pelawat-pakai-niqab-di-tempat-awam

...

Got article: polis-tangkap-22-suspek-sindiket-penipuan-pelaburan-saham
Got article: pdrm-colombia-kerjasama-dadah-rentas-sempadan
Found existing article: semua-keputusan-ditentu-majlis-presiden-ph-kabinet
Got article: agil-natt-dilantik-pengerusi-affin
Got article: bakal-suami-peluk-islam-dengan-rela-hati-sarah
Found existing article: sic-tingkat-sistem-keselamatan-elak-kes-kecurian-berulang
Got article: salzburg-tangguh-kemaraan-napoli
Found existing article: hakim-timbalan-pendakwa-raya-peguam-hadapi-tuduhan-bersama-terima-suapan
Found existing article: kalau-azmin-boleh-jumpa-anwar-hadirlah-mesyuarat-parti-abdullah-sani
Found existing article: amanah-pegang-konsensus-salahuddin
Got article: rsi-international-diserbu-guna-perkataan-bank-tanpa-kelulusan
Got article: 237-mangsa-banjir-masih-berada-di-dua-pps-di-perak
Got article: permohonan-ke-kolej-matrikulasi-ipg-dibuka-bulan-ini
Found existing article: peluang-jdt-rangkul-treble
Got article: saham-asia-meningkat-di-tengah-tengah-per

Got article: mtdc-buka-pusat-pencetakan-3d-untuk-pks
Got article: sikap-pengotor-cemari-imej-negeri-cantik
Got article: bahasa-melayu-biarlah-biasa-biasa-tapi-berbisa
Got article: dewan-rakyat-ucap-selamat-ulang-tahun-kelahiran-ke-94-kepada-pm
Got article: malaysia-terima-nota-permohonan-ekstradisi-dr-zakir-naik
Found existing article: pindahkan-249-tahanan-kanak-kanak-ke-pusat-perlindungan
Got article: waris-bekas-penghuni-gesa-jkm-telus-jalankan-siasatan
Got article: kedah-tidak-pandang-mudah-felda-united
Found existing article: omcorp-sasar-jualan-rm13-juta-tahun-depan
Found existing article: real-madrid-ikat-messi-jepun
Got article: gempa-bumi-65-landa-chile
Got article: remaja-lelaki-maut-akibat-strok-haba-di-sepanyol
Got article: lima-warga-china-didakwa-dalam-insiden-bangunan-runtuh
Got article: neymar-perlu-mohon-maaf
Got article: suhu-bakal-cecah-68-darjah-di-kuwait-seorang-maut-kepanasan-melampau
Found existing article: tiga-sekeluarga-maut-4-lagi-cedera-2-kenderaan-bertembun

Got article: situasi-di-pusat-rawatan-awal-terus-sibuk
Got article: tiada-rombakan-exco-perak
Found existing article: dbp-pendakwah-bahasa-melayu-lebih-berkesan
Got article: tidak-gentar-saingi-jenama-antarabangsa
Got article: mesyuarat-kabinet-esok-tentukan-perintah-darurat
Found existing article: warga-emas-jatuh-motosikal-diseret-lori-80-meter
Got article: 720-pelajar-di-los-angeles-adalah-gelandangan
Got article: keputusan-serie
Found existing article: persada-keris-mas-ukm-dirasmikan
Got article: polis-buru-dalang-kes-bunuh-usahawan-indonesia
Got article: bayi-4-bulan-meninggal-dunia-di-taska
Got article: nasib-dovizioso-ducati-sebelum-gp-argentina
Got article: tiada-mangsa-terselamat-nahas-pesawat-ethiopian-airlines
Got article: 7-ahli-geng-08-terbabit-kes-pemerasan-tapak-pembinaan-dicekup
Got article: jangan-jadikan-usas-medan-pertempuran-politik-ph-saarani
Got article: pemeriksaan-pasport-jemaah-haji-malaysia-dilaksana-lebih-awal
Got article: laksana-sst-dalam-tempoh-singkat-ja

Got article: kertas-cadangan-elaun-kritikal-bomba-perlu-dirujuk-kepada-menteri-ksn
Got article: gasly-verstappen-pantas-dalam-hujan
Got article: potek-idam-sertai-skuad-kebangsaan
Got article: juruelektrik-bantu-curi-elektrik
Got article: sokong-pm-untuk-kestabilan-pentadbiran-ekonomi-azmin
Got article: bayern-dortmund-rebut-piala-super-jerman
Got article: kandungan-sebenar-ghost-smoke-diketahui-minggu-depan
Got article: rakyat-mahu-lihat-prestasi-ph
Found existing article: populasi-harimau-malaya-berada-di-angka-darurat
Got article: tiada-cukai-baharu-dalam-belanjawan-2020
Got article: pengalaman-mengajar-gps-selesa-bergerak-sendiri
Got article: keadaan-statik-meninggikan-risiko-kecederaan-pada-adib
Got article: kerajaan-selangor-buka-permohonan-hpipt
Got article: video-lucah-haziq-lima-individu-direman-6-hari
Found existing article: griezmann-seronok-dapat-sertai-barcelona
Found existing article: umno-nazri-aziz-tak-langgar-disiplin-parti
Got article: malaysia-ke-35-indeks-inovasi-gl

Got article: seorang-maut-14-cedera-kemalangan
Found existing article: 60-petani-etnik-fulani-dibunuh-di-mali
Got article: tiada-asas-salahkan-ytl
Got article: 60-pesalah-dadah-ditahan
Got article: bekas-ketua-nissan-ghosn-ditangkap-kali-keempat
Got article: whc-2019-platform-lonjak-industri-halal-global
Got article: malaysia-contoh-negara-perjuang-keamanan-sejagat
Got article: niger-rampas-10-tan-ubat-dan-produk-kecantikan-tiruan
Got article: siri-pertama-myrumah-2019-tawar-rumah-serendah-rm130000
Got article: united-ada-keyakinan-untuk-menang%C2%A0-solskjaer
Got article: jokowi-enggan-batal-lawatan-ke-sibolga
Got article: pengusiran-penyewa-ppr-taman-manggis-didakwa-tidak-sah
Got article: juruterbang-ditawan-pakistan-pulang-ke-india
Got article: latiff-teruja-dipanggil-lagi-perkuat-harimau-malaya
Got article: pkr-kutuk-serangan-pengganas-di-new-zealand
Got article: bumiputra-rockers-sedia-ambulans
Found existing article: wartawan-bh-raih-5-anugerah-media-johor
Got article: cuma-enam-